In [214]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff


#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm 

from sklearn import model_selection as ms
from sklearn.datasets import make_regression

In [268]:
#data = np.loadtxt('datasets/cocomonasa_v1.csv', delimiter=",")
data, meta = arff.loadarff('datasets/cocomonasa_v1.arff')
df = pd.DataFrame(data)
df.head()

,RELY,DATA,CPLX,TIME,STOR,VIRT,TURN,ACAP,AEXP,PCAP,VEXP,LEXP,MODP,TOOL,SCED,LOC,ACT_EFFORT
0,b'Nominal',b'High',b'Very_High',b'Nominal',b'Nominal',b'Low',b'Nominal',b'High',b'Nominal',b'Very_High',b'Low',b'Nominal',b'High',b'Nominal',b'Low',70.0,278.0
1,b'Very_High',b'High',b'High',b'Very_High',b'Very_High',b'Nominal',b'Nominal',b'Very_High',b'Very_High',b'Very_High',b'Nominal',b'High',b'High',b'High',b'Low',227.0,1181.0
2,b'Nominal',b'High',b'High',b'Very_High',b'High',b'Low',b'High',b'High',b'Nominal',b'High',b'Low',b'High',b'High',b'Nominal',b'Low',177.9,1248.0
3,b'High',b'Low',b'High',b'Nominal',b'Nominal',b'Low',b'Low',b'Nominal',b'Nominal',b'Nominal',b'Nominal',b'High',b'High',b'Nominal',b'Low',115.8,480.0
4,b'High',b'Low',b'High',b'Nominal',b'Nominal',b'Low',b'Low',b'Nominal',b'Nominal',b'Nominal',b'Nominal',b'High',b'High',b'Nominal',b'Low',29.5,120.0


In [269]:
#adotando escala de atributos
#Ex. {'Very_low', 'Low', 'Nominal', 'High', 'Extra_High'} => {'1', '2', '3','4','5'}
tranform_data = {"Very_Low":"1", "Low":"2", "Nominal":"4" , "High":"5" ,"Very_High":"5", "Extra_High":"6"}

df_transform = df
#df_transform = df_transform.replace(tranform_data)

df_tranform.head()

#df.head() #3, 5, 3, 4, 4


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,b'Nominal',b'High',b'Very_High',b'Nominal',b'Nominal',b'Low',b'Nominal',b'High',b'Nominal',b'Very_High',b'Low',b'Nominal',b'High',b'Nominal',b'Low',70.0,278.0
1,b'Very_High',b'High',b'High',b'Very_High',b'Very_High',b'Nominal',b'Nominal',b'Very_High',b'Very_High',b'Very_High',b'Nominal',b'High',b'High',b'High',b'Low',227.0,1181.0
2,b'Nominal',b'High',b'High',b'Very_High',b'High',b'Low',b'High',b'High',b'Nominal',b'High',b'Low',b'High',b'High',b'Nominal',b'Low',177.9,1248.0
3,b'High',b'Low',b'High',b'Nominal',b'Nominal',b'Low',b'Low',b'Nominal',b'Nominal',b'Nominal',b'Nominal',b'High',b'High',b'Nominal',b'Low',115.8,480.0
4,b'High',b'Low',b'High',b'Nominal',b'Nominal',b'Low',b'Low',b'Nominal',b'Nominal',b'Nominal',b'Nominal',b'High',b'High',b'Nominal',b'Low',29.5,120.0


In [267]:
def Normalized(X):
    #TODO: NORMALIZAR OS DADOS USANDO Z_SCORE
     X_Norm = X
     for i in range(len(X[0])):
        m = np.mean(X[:, i])
        s = np.std(X[:,i])
        X_Norm[:, i] = (X[:,i] - m) / s
     return X_Norm

In [250]:
#preparando dados
#data, meta = arff.loadarff('datasets/cocomo81.arff')
#data = np.loadtxt('datasets/cocomo81.csv', delimiter=",")
#Normalizando os valores
data = Normalized(data)
#removendo o atributo classe dos dados.
X = data[:, :-1]
#armazenando o atributo classe.
y = data[:,-1]

#df = pd.DataFrame(data)
#df.head()
#df.hist(bins=50,figsize=(20,15))
#plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [16]:
models = []
#Instanciando os modelos
models.append(['KNN', KNeighborsRegressor(n_neighbors=5)]) 
models.append(['MLP', MLPRegressor(activation='logistic', solver='sgd', alpha=0.02, max_iter=300, hidden_layer_sizes=200)])
models.append(['RFR', RandomForestRegressor()])
models.append(['SVR', svm.SVR()])

In [20]:
for model in models:
    cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)
    scores = ms.cross_val_score(model[1], X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    #converter para positivo.
    scores = np.absolute(scores)
    s_mean = np.mean(scores)
    s_std = np.std(scores)
    
    print('-', model[0],'------------------------------------')
    print('Scores:', scores, 'MAE: %.4f' % (s_mean), 'STD: %.4f' % (s_std)) 

- KNN ------------------------------------
Scores: [0.14698373 0.58256974 0.25355471] MAE: 0.3277 STD: 0.1854
- MLP ------------------------------------
Scores: [0.29216461 0.55301983 0.37739488] MAE: 0.4075 STD: 0.1086
- RFR ------------------------------------
Scores: [0.17865556 0.53352765 0.29111612] MAE: 0.3344 STD: 0.1481
- SVR ------------------------------------
Scores: [0.19270733 0.58631987 0.36448093] MAE: 0.3812 STD: 0.1611


/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
